# GPT 5

This notebook implements a process to generate and evaluate Plain Language Summaries (PLS) of biomedical abstracts using GPT-5. Here's a brief overview of the steps involved:


1. Setup and Authentication: Installs necessary libraries and sets up the OpenAI API key.
Data Ingestion: Downloads biomedical abstracts and their corresponding ground truth Plain Language Summaries (PLS) from a GitHub repository.

2. Text Simplification: Utilizes the GPT-5 model to generate simplified versions of the downloaded abstracts based on a detailed prompt for PLS creation.
3. Data Preparation: Organizes the original abstracts, GPT-5 generated simplified texts, and ground truth PLS texts into a Pandas DataFrame.
4. Evaluation: Calculates several metrics to assess the quality of the GPT-5 generated simplified texts:

+ BERTScore: Measures the semantic similarity (relevance) between the generated simplified texts and the ground truth PLS.
+ AlignScore: Assesses the factual consistency between the generated simplified texts and the ground truth PLS.
+ Readability Scores: Computes various readability indices (e.g., Flesch-Kincaid Grade Level, Flesch Reading Ease) to determine the ease of understanding of the generated simplified texts.


Finally, all the texts and their computed scores are saved to CSV files, and the average scores are printed.

## Setup and authentication

Install necessary libraries and get OPENAI_API_KEY from os


In [ ]:
%pip install torch evaluate[bertscore] transformers sentence-transformers matplotlib pandas numpy textstat

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import torch
print(torch.__version__)

2.8.0+cu126


## Data Ingestion - Cochrane
Load text files from the GitHub repository "https://api.github.com/repos/feliperussi/bridging-the-gap-in-health-literacy/".

In [ ]:
%pip install requests

Load abstracts



In [ ]:
import csv

file_names = []
with open('/content/abstract_names.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        file_names.append(row[0]) # Assuming each row contains one filename

print(f"Loaded {len(file_names)} filenames from abstract_names.csv")

Loaded 300 filenames from abstract_names.csv


In [ ]:
import requests
import os

github_base_urls = [
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/",
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/test/non_pls/",
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/train/non_pls/"
]
local_download_dir = 'downloaded_texts'

# Create the local directory if it doesn't exist
os.makedirs(local_download_dir, exist_ok=True)

downloaded_count = 0

for file_name in file_names:
    local_file_path = os.path.join(local_download_dir, file_name)
    downloaded = False

    if os.path.exists(local_file_path):
        print(f"File {file_name} already exists locally. Skipping download.")
        downloaded_count += 1
        downloaded = True
        continue # Skip to the next file_name if already downloaded


    for base_url in github_base_urls:
        file_url = base_url + file_name

        try:
            print(f"Attempting to download {file_name} from {base_url}")
            file_content_response = requests.get(file_url)
            file_content_response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

            with open(local_file_path, 'w', encoding='utf-8') as f:
                f.write(file_content_response.text)
            downloaded_count += 1
            downloaded = True
            print(f"Successfully downloaded {file_name}")
            break  # Move to the next file_name if download is successful

        except requests.exceptions.RequestException as e:
            print(f"Could not download {file_name} from {base_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {file_name} from {base_url}: {e}")

    if not downloaded:
        print(f"Warning: Could not download {file_name} from any of the provided URLs.")


print(f"Download complete. Successfully downloaded {downloaded_count} out of {len(file_names)} files.")

Attempting to download 10.1002-14651858.CD000371.pub7-abstract.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/
Successfully downloaded 10.1002-14651858.CD000371.pub7-abstract.txt
Attempting to download 10.1002-14651858.CD001218.pub3-abstract.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/
Successfully downloaded 10.1002-14651858.CD001218.pub3-abstract.txt
Attempting to download 10.1002-14651858.CD001977.pub2-abstract.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/
Successfully downloaded 10.1002-14651858.CD001977.pub2-abstract.txt
Attempting to download 10.1002-14651858.CD002201.pub6-abstract.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/
Successfully dow

### Load abstract text

Load the content of the downloaded text files into a dictionary.

In [ ]:
import os

downloaded_texts_content = {}

for filename in os.listdir(local_download_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(local_download_dir, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                downloaded_texts_content[filename] = f.read()
            print(f"Loaded text from {filename}")
        except Exception as e:
            print(f"Error loading text from {filename}: {e}")


Loaded text from 10.1002-14651858.CD013170.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD004957.pub3-abstract.txt
Loaded text from 10.1002-14651858.CD013717.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013384.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013674.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD006251.pub4-abstract.txt
Loaded text from 10.1002-14651858.CD013285.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD015397-abstract.txt
Loaded text from 10.1002-14651858.CD003552.pub4-abstract.txt
Loaded text from 10.1002-14651858.CD013699-abstract.txt
Loaded text from 10.1002-14651858.CD013515.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013251.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013190-abstract.txt
Loaded text from 10.1002-14651858.CD012979.pub3-abstract.txt
Loaded text from 10.1002-14651858.CD013664.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013196.pub2-abstract.txt
Loaded text from 10.1002-14651858.CD013

In [ ]:
print("Downloaded filenames:")
for filename in downloaded_texts_content.keys():
    print(filename)

Downloaded filenames:
10.1002-14651858.CD013170.pub2-abstract.txt
10.1002-14651858.CD004957.pub3-abstract.txt
10.1002-14651858.CD013717.pub2-abstract.txt
10.1002-14651858.CD013384.pub2-abstract.txt
10.1002-14651858.CD013674.pub2-abstract.txt
10.1002-14651858.CD006251.pub4-abstract.txt
10.1002-14651858.CD013285.pub2-abstract.txt
10.1002-14651858.CD015397-abstract.txt
10.1002-14651858.CD003552.pub4-abstract.txt
10.1002-14651858.CD013699-abstract.txt
10.1002-14651858.CD013515.pub2-abstract.txt
10.1002-14651858.CD013251.pub2-abstract.txt
10.1002-14651858.CD013190-abstract.txt
10.1002-14651858.CD012979.pub3-abstract.txt
10.1002-14651858.CD013664.pub2-abstract.txt
10.1002-14651858.CD013196.pub2-abstract.txt
10.1002-14651858.CD013501.pub2-abstract.txt
10.1002-14651858.CD013172.pub2-abstract.txt
10.1002-14651858.CD013679-abstract.txt
10.1002-14651858.CD007263.pub3-abstract.txt
10.1002-14651858.CD013829.pub2-abstract.txt
10.1002-14651858.CD013512.pub2-abstract.txt
10.1002-14651858.CD013487-abst

In [ ]:
print(len(downloaded_texts_content))

300


In [ ]:
import pandas as pd

# Create a DataFrame from the dictionary
# Assuming the dictionary keys are filenames and values are text content
df_downloaded = pd.DataFrame.from_dict(downloaded_texts_content, orient='index', columns=['text_content'])

# Save the DataFrame to a CSV file
csv_filepath = 'downloaded_texts_content.csv'
df_downloaded.to_csv(csv_filepath, index_label='filename')

print(f"Downloaded texts content saved to {csv_filepath}")

Downloaded texts content saved to downloaded_texts_content.csv


## Text Simplification

Use the GPT 5 with the provided prompt to simplify the extracted text.

In [ ]:
import time
from openai import OpenAI
client = OpenAI()


simplified_texts = {}
prompt = """Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description of the design,
description of study and patient population (age, health condition, gender), and the expected
amount of time a person will be in the study.
4. Results: Answer ‘What were the main results of the study’, include the benefits for the patients,
how the study was relevant for the area of study, and the conclusions from the investigator.
c. Consistency and Replicability: The generated PLS should be consistent regardless of the order of
sentences or the specific phrasing used in the input protocol text.
d. Compliance with Plain Language Guidelines: The generated PLS must follow all these plain
language guidelines:
• Have readability grade level of 6 or below.
• Do not have jargon. All technical or medical words or terms should be defined or broken down
into simple and logical explanations.
• Active voice, not passive.
• Mostly one or two syllable words.
• Sentences of 15 words or less.
• Short paragraphs of 3-5 sentences.
• Simple numbers (e.g., ratios, no percentages).
e. Do not invent Content: The AI model should not invent information. If the AI model includes data
other than the one given in the input abstract, the AI model should guarantee such data is verified and
real.
f. Aim for an approximate PLS length of 500-900 words.


Abstract of a biomedical study text: {text}
"""

counter_new_run = 0
sum_times_new_run = 0

for filename in downloaded_texts_content.keys:
    counter_new_run += 1
    text = downloaded_texts_content[filename]
    retries = 3  # Set a number of retries

    while retries > 0:
        init_time = time.time()

        try:
            response = client.responses.create(
                model="gpt-5",
                input=prompt.format(text=text)
                )

            simplified_texts[filename] = response.output_text
            response_time = time.time() - init_time
            sum_times_new_run += response_time
            print(f"Simplified text # {len(simplified_texts)} for {filename}. Duration: {response_time}")
            break  # Exit the retry loop on success

        except Exception as e:
            print(f"An unexpected error occurred while simplifying {filename}: {e}")
            retries -= 1
            if retries > 0:
                print(f"Retrying {filename}, {retries} retries left...")
                time.sleep(5) # Wait before retrying
            else:
                print(f"Failed to simplify {filename} after multiple retries.")
                break # Exit the retry loop after all retries are exhausted

if counter_new_run > 0:
    print(f"Average time for new runs: {sum_times_new_run/counter_new_run}")
else:
    print("No new files were processed in this run.")

print(f"Total simplified texts: {len(simplified_texts)}")

Continuing simplification for 42 remaining files...
Simplified text # 259 for 10.1002-14651858.CD013180.pub2-abstract.txt. Duration: 80.49999952316284
Simplified text # 260 for 10.1002-14651858.CD007077.pub4-abstract.txt. Duration: 69.96734523773193
Simplified text # 261 for 10.1002-14651858.CD009868.pub3-abstract.txt. Duration: 68.58440041542053
Simplified text # 262 for 10.1002-14651858.CD013705.pub3-abstract.txt. Duration: 114.07494688034058
Simplified text # 263 for 10.1002-14651858.CD013298.pub2-abstract.txt. Duration: 61.27224159240723
Simplified text # 264 for 10.1002-14651858.CD013390.pub2-abstract.txt. Duration: 91.75086450576782
Simplified text # 265 for 10.1002-14651858.CD009134.pub2-abstract.txt. Duration: 68.92131495475769
Simplified text # 266 for 10.1002-14651858.CD013217.pub2-abstract.txt. Duration: 76.75734210014343
Simplified text # 267 for 10.1002-14651858.CD013757.pub2-abstract.txt. Duration: 73.17500925064087
Simplified text # 268 for 10.1002-14651858.CD012204.pub2

In [ ]:
import pandas as pd

# Create a DataFrame from the extracted and simplified texts
data = {'original_text': downloaded_texts_content, 'simplified_text': simplified_texts}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_filepath = 'simplified_texts_gpt5.csv'
df.to_csv(csv_filepath, index_label='filename')

print(f"Results saved to {csv_filepath}")

Results saved to simplified_texts_gpt5_1.csv


# Task
Load original pls and compare to simplified texts from "/content/simplified_texts.csv" and provide a Relevance score using BERTScore, Factual consistence score using AlignScore and Readability score using Flesch–Kincaid Grade Level for each row.

## Load pls text

Load pls

In [ ]:
import csv

pls_file_names = []
with open('/content/pls_names.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        pls_file_names.append(row[0]) # Assuming each row contains one filename

print(f"Loaded {len(pls_file_names)} filenames from pls_names.csv")

Loaded 300 filenames from pls_names.csv


In [ ]:
import requests
import os

github_base_urls = [
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/",
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/test/pls/",
    "https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/train/pls/"
]
local_download_dir = 'pls_downloaded_texts'

# Create the local directory if it doesn't exist
os.makedirs(local_download_dir, exist_ok=True)

downloaded_count = 0

for file_name in pls_file_names:
    local_file_path = os.path.join(local_download_dir, file_name)
    downloaded = False

    if os.path.exists(local_file_path):
        print(f"File {file_name} already exists locally. Skipping download.")
        downloaded_count += 1
        downloaded = True
        continue # Skip to the next file_name if already downloaded


    for base_url in github_base_urls:
        file_url = base_url + file_name

        try:
            print(f"Attempting to download {file_name} from {base_url}")
            file_content_response = requests.get(file_url)
            file_content_response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

            with open(local_file_path, 'w', encoding='utf-8') as f:
                f.write(file_content_response.text)
            downloaded_count += 1
            downloaded = True
            print(f"Successfully downloaded {file_name}")
            break  # Move to the next file_name if download is successful

        except requests.exceptions.RequestException as e:
            print(f"Could not download {file_name} from {base_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {file_name} from {base_url}: {e}")

    if not downloaded:
        print(f"Warning: Could not download {file_name} from any of the provided URLs.")


print(f"Download complete. Successfully downloaded {downloaded_count} out of {len(pls_file_names)} files.")

Attempting to download 10.1002-14651858.CD000371.pub7-pls.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/
Could not download 10.1002-14651858.CD000371.pub7-pls.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/llms_testing/Cochrane/ground_truth/10.1002-14651858.CD000371.pub7-pls.txt
Attempting to download 10.1002-14651858.CD000371.pub7-pls.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/test/pls/
Could not download 10.1002-14651858.CD000371.pub7-pls.txt from https://raw.githubusercontent.com/feliperussi/bridging-the-gap-in-health-literacy/main/data_collection_and_processing/Data%20Sources/Cochrane/t

In [ ]:
import os

pls_downloaded_texts_content = {}

for filename in os.listdir(local_download_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(local_download_dir, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                clean_name = filename.replace("-pls.txt", "")
                pls_downloaded_texts_content[clean_name] = f.read()
            print(f"Loaded text from {filename}")
        except Exception as e:
            print(f"Error loading text from {filename}: {e}")


Loaded text from 10.1002-14651858.CD011931.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013697.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013243.pub2-pls.txt
Loaded text from 10.1002-14651858.CD012204.pub2-pls.txt
Loaded text from 10.1002-14651858.CD006237.pub4-pls.txt
Loaded text from 10.1002-14651858.CD012129.pub3-pls.txt
Loaded text from 10.1002-14651858.CD010497.pub2-pls.txt
Loaded text from 10.1002-14651858.CD014963-pls.txt
Loaded text from 10.1002-14651858.CD013256.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013432.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013438-pls.txt
Loaded text from 10.1002-14651858.CD006682.pub6-pls.txt
Loaded text from 10.1002-14651858.CD013829.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013397.pub2-pls.txt
Loaded text from 10.1002-14651858.CD010852.pub2-pls.txt
Loaded text from 10.1002-14651858.CD013224.pub2-pls.txt
Loaded text from 10.1002-14651858.CD003147.pub5-pls.txt
Loaded text from 10.1002-14651858.CD011357.pub2-pls.txt
Lo

In [ ]:
print(pls_downloaded_texts_content)

{'10.1002-14651858.CD011931.pub2': "Preventing pain after brain surgery\nThe problem \nThere is increasing evidence that people who have undergone brain surgery experience significant pain. This pain can have serious consequences including raised blood pressure, agitation, prolonged recovery time and an increased risk of long‐term headaches. Research studies have looked at different drugs in an attempt to reduce the risk of pain for these people. There is now more evidence about pain reduction options for adults undergoing brain surgery but there remains uncertainty as to which options work best. \nThe question \nThis review aimed to determine which drugs provide the best chance of reducing pain for adults undergoing brain surgery, by collecting and combining the results of studies that looked at pain‐relieving drugs for this patient group. To provide an accurate answer to this question, only studies conducted in accordance with an approved high standard were included. Studies publishe

In [ ]:
import pandas as pd

# Load the simplified_texts.csv file
df_simplified = pd.read_csv('simplified_texts_gpt5.csv')

# Remove "-abstract.txt" from the 'filename' column
df_simplified['filename'] = df_simplified['filename'].str.replace('-abstract.txt', '', regex=False)

# Set 'filename' as the index for both dataframes for merging
df_simplified = df_simplified.set_index('filename')
df_pls = pd.DataFrame.from_dict(pls_downloaded_texts_content, orient='index', columns=['pls_text_content'])


# Merge the two dataframes
df_merged = df_simplified.join(df_pls)

# Display the head of the merged DataFrame
display(df_merged.head())

,original_text,simplified_text,pls_text_content
filename,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...


In [ ]:
display(df_merged.head())
display(df_merged.info())

,original_text,simplified_text,pls_text_content
filename,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...


<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 10.1002-14651858.CD013170.pub2 to 10.1002-14651858.CD015270
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   original_text     300 non-null    object
 1   simplified_text   300 non-null    object
 2   pls_text_content  300 non-null    object
dtypes: object(3)
memory usage: 17.5+ KB


None

In [ ]:
# Save the DataFrame to a CSV file
csv_filepath = 'abstract_generated_pls_gpt5.csv'
df_merged.to_csv(csv_filepath, index_label='filename')

print(f"DataFrame saved to {csv_filepath}")

DataFrame saved to abstract_generated_pls_gpt5.csv


## Load data

Load the data from "/content/abstract_generated_pls_gpt5.csv" into a pandas DataFrame.


In [ ]:
import pandas as pd

df = pd.read_csv("abstract_generated_pls_gpt5.csv", index_col='filename')
display(df.head())
display(df.info())

,original_text,simplified_text,pls_text_content
filename,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...


<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 10.1002-14651858.CD013170.pub2 to 10.1002-14651858.CD015270
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   original_text     300 non-null    object
 1   simplified_text   300 non-null    object
 2   pls_text_content  300 non-null    object
dtypes: object(3)
memory usage: 9.4+ KB


None

# Calculate scores

## Calculate bertscore

Compute BERTScore for each pair of original and simplified texts.


In [ ]:
%pip install evaluate[bertscore] transformers sentence-transformers matplotlib pandas numpy textstat

In [ ]:
%pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

precision_scores = []
recall_scores = []
f1_scores = []

for index, row in df.iterrows():
    original_text = row['pls_text_content']
    simplified_text = row['simplified_text']

    # BERTScore expects lists of strings
    results = bertscore.compute(predictions=[simplified_text], references=[original_text], model_type="allenai/longformer-large-4096-finetuned-triviaqa")

    f1_scores.append(results['f1'][0])

df['bertscore'] = f1_scores

display(df.head())

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


,original_text,simplified_text,pls_text_content,bertscore
filename,,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...,0.813240
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...,0.804248
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...,0.846057
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...,0.829253
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...,0.821670


In [ ]:
print(sum(df['bertscore']) / len(df['bertscore']))

0.832703308860461


## Calculate AlignScore

Compute AlignScore for each pair of original and simplified texts.

In [ ]:
# Clone the alignscore repository
!pip install git+https://github.com/yuh-zha/AlignScore.git --no-deps
!pip install pytorch-lightning==1.9.5

import torch
import transformers
if not hasattr(transformers, "AdamW"):
    from torch.optim import AdamW
    transformers.AdamW = AdamW


  Cloning https://github.com/yuh-zha/AlignScore.git to /tmp/pip-req-build-4y5t6__x
  Running command git clone --filter=blob:none --quiet https://github.com/yuh-zha/AlignScore.git /tmp/pip-req-build-4y5t6__x
  Resolved https://github.com/yuh-zha/AlignScore.git to commit a0936d5afee642a46b22f6c02a163478447aa493
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for alignscore: filename=alignscore-0.1.3-py3-none-any.whl size=18369 sha256=97fddaa7c81702c0c9870c7772cf3f383060db904bdafa4493ecc4d8f0bace49
  Stored in directory: /tmp/pip-ephem-wheel-cache-8zivlak4/wheels/61/78/cf/7adbff432a586bfb8a1673348218105047a4845e89a536007d
Successfully built alignscore
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 73.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
from alignscore import AlignScore

In [ ]:
# Download the spaCy language model
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 137.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Install textstat if not already installed
%pip install textstat

import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import torch

torch.cuda.empty_cache()
print("CUDA cache cleared.")

CUDA cache cleared.


In [ ]:
device_id = 'cuda' if torch.cuda.is_available() else 'cpu'

alignscorer = AlignScore(model='roberta-large',
    batch_size=32,
    device=device_id,
    ckpt_path='https://huggingface.co/yzha/AlignScore/resolve/main/AlignScore-large.ckpt',
    evaluation_mode='bin_sp',
    verbose=False
)

align_scores = []

for index, row in df.iterrows():
    original_text = row['pls_text_content']
    simplified_text = row['simplified_text']
    # Pass texts as lists to the score method
    align_scores.append(alignscorer.score([original_text], [simplified_text])[0])
    print(align_scores[-1])

df['alignscore'] = align_scores

display(df.head())

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading: "https://huggingface.co/yzha/AlignScore/resolve/main/AlignScore-large.ckpt" to /root/.cache/torch/hub/checkpoints/AlignScore-large.ckpt


100%|██████████| 4.56G/4.56G [00:14<00:00, 345MB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.0.post1 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file https:/huggingface.co/yzha/AlignScore/resolve/main/AlignScore-large.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['base_model.embeddings.position_ids']
  rank_zero_warn(


...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

0.5239907503128052
0.5924710035324097
0.6066861152648926
0.6068457961082458
0.5349065065383911
0.608736515045166
0.6482164263725281
0.4912967085838318
0.6094803810119629
0.4756123423576355
0.573861837387085
0.5260696411132812
0.514022171497345
0.5188335180282593
0.576082706451416
0.46946945786476135
0.556978702545166
0.4271763563156128
0.5719823241233826
0.6227624416351318
0.4885163903236389
0.42554745078086853
0.5402676463127136
0.43092092871665955
0.5643678307533264
0.43790680170059204
0.5866411924362183
0.6059098243713379
0.6016711592674255
0.4616034924983978
0.6621822118759155
0.4943959712982178
0.7098910212516785
0.504234254360199
0.7069680690765381
0.5498420596122742
0.6748560070991516
0.600536048412323
0.5080739855766296
0.616641640663147
0.4356323480606079
0.6333791613578796
0.5865247249603271
0.6226927638053894
0.5975991487503052
0.479241281747818
0.3707161247730255
0.4629049599170685
0.4850617051124573
0.5534427762031555
0.5456685423851013
0.5454806089401245
0.568063318729400

,original_text,simplified_text,pls_text_content,bertscore,alignscore
filename,,,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...,0.813240,0.523991
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...,0.804248,0.592471
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...,0.846057,0.606686
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...,0.829253,0.606846
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...,0.821670,0.534907


In [ ]:
print(sum(df['alignscore']) / len(df['alignscore']))

0.5501166886091232


## Calculate readability scores

In [ ]:
!pip install https://github.com/andreasvc/readability/tarball/master
!pip install syntok

     - 35.0 kB 10.6 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36388 sha256=23999f7afe48f5add9f17b8a528383bea58965ec7fa89a961b4a1c04fe081bdb
  Stored in directory: /tmp/pip-ephem-wheel-cache-3x_t26cv/wheels/b9/ca/f5/acdb922de5c90229d7290b12a5b7610a2d44855a0d70b72b48
Successfully built readability


In [ ]:
import readability
import syntok.segmenter as segmenter


flesch_kincaid_scores = []
coleman_liau_scores = []
flesch_reading_ease_scores = []
gunning_fog_scores = []
smog_index_scores = []
dale_chall_scores = []



for index, row in df.iterrows():
    simplified_text = row['simplified_text']
    tokenized = '\n\n'.join(
     '\n'.join(' '.join(token.value for token in sentence)
        for sentence in paragraph)
     for paragraph in segmenter.analyze(simplified_text))

    results = readability.getmeasures(tokenized, lang='en')

    # Calculate Flesch-Kincaid Grade Level
    flesch_kincaid_scores.append(results['readability grades']['Kincaid'])

    # Calculate Coleman-Liau Index
    coleman_liau_scores.append(results['readability grades']['Coleman-Liau'])

    # Calculate Flesch Reading Ease
    flesch_reading_ease_scores.append(results['readability grades']['FleschReadingEase'])

    # Calculate Gunning Fog Index
    gunning_fog_scores.append(results['readability grades']['GunningFogIndex'])

    # Calculate SMOG Index
    smog_index_scores.append(results['readability grades']['SMOGIndex'])

    # Calculate Dale-Chall Readability Score
    dale_chall_scores.append(results['readability grades']['DaleChallIndex'])

df['flesch_kincaid_grade'] = flesch_kincaid_scores
df['coleman_liau_index'] = coleman_liau_scores
df['flesch_reading_ease'] = flesch_reading_ease_scores
df['gunning_fog_index'] = gunning_fog_scores
df['smog_index'] = smog_index_scores
df['dale_chall_score'] = dale_chall_scores


display(df.head())

,original_text,simplified_text,pls_text_content,bertscore,alignscore,flesch_kincaid_grade,coleman_liau_index,flesch_reading_ease,gunning_fog_index,smog_index,dale_chall_score
filename,,,,,,,,,,,
10.1002-14651858.CD013170.pub2,Background\nPeople with neuromuscular disorder...,Plain Title\nDo cough help methods work and ar...,The safety and effectiveness of techniques to ...,0.813240,0.523991,0.982321,4.450507,102.337206,4.172322,5.401922,3.219250
10.1002-14651858.CD004957.pub3,Multidisciplinary rehabilitation programmes fo...,Plain Title\nTeam care after hip or knee joint...,Multidisciplinary rehabilitation programmes fo...,0.804248,0.592471,2.302598,5.398012,94.117604,5.348099,6.484660,3.918396
10.1002-14651858.CD013717.pub2,"Background\nIn late 2019, the first cases of c...",Plain Title\nDo travel rules like border check...,Can international travel‐related control measu...,0.846057,0.606686,3.740024,7.153470,84.877878,5.876737,6.872983,4.821197
10.1002-14651858.CD013384.pub2,Background\nChronic obstructive pulmonary dise...,Plain Title\nDo COPD care plans that fit other...,Approaches to help people with COPD who have o...,0.829253,0.606846,3.208390,6.475167,92.346153,6.106585,6.639609,4.111823
10.1002-14651858.CD013674.pub2,Background\nMajor depressive disorders have a ...,1. Plain Title\nHow well do newer antidepressa...,Newer generation antidepressants for depressio...,0.821670,0.534907,3.904956,7.683775,83.291718,6.973268,7.769408,4.737436


In [ ]:
# Save the DataFrame to a CSV file
csv_filepath = 'simplified_texts_with_scores_gpt5.csv'
df.to_csv(csv_filepath, index_label='filename')

print(f"DataFrame saved to {csv_filepath}")

DataFrame saved to simplified_texts_with_scores_gpt5.csv


In [ ]:
print(sum(df['bertscore']) / len(df['bertscore']))
print(sum(df['alignscore']) / len(df['alignscore']))
print(sum(df['flesch_reading_ease']) / len(df['flesch_reading_ease']))
print(sum(df['flesch_kincaid_grade']) / len(df['flesch_kincaid_grade']))
print(sum(df['coleman_liau_index']) / len(df['coleman_liau_index']))
print(sum(df['gunning_fog_index']) / len(df['gunning_fog_index']))
print(sum(df['smog_index']) / len(df['smog_index']))
print(sum(df['dale_chall_score']) / len(df['dale_chall_score']))


0.832703308860461
0.5501166886091232
94.70572683718869
2.324973807412397
5.840141870164148
5.375416283200641
6.327062887232856
4.06746028096577
